<a href="https://colab.research.google.com/github/BrunoDkno368/integracion-de-varias-fuentes/blob/main/Distintas_fuentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ARQUITECTURA DEL PROYECTO

FUENTES → STAGING → LIMPIEZA → REGLAS DE NEGOCIO → DATASET FINAL → DASHBOARD

In [214]:
import pandas as pd

In [215]:
ventas_online = pd.read_csv('/content/drive/MyDrive/Varias fuentes/ventas_online.csv')
stock = pd.read_csv('/content/drive/MyDrive/Varias fuentes/stock_productos.csv')
cotizacion =pd.read_csv('/content/drive/MyDrive/Varias fuentes/cotizacion_dolar.csv')
cliente= pd.read_json('/content/drive/MyDrive/Varias fuentes/clientes.json')
ventas_tienda = pd.read_excel('/content/drive/MyDrive/Varias fuentes/ventas_tienda.xlsx')


##NORMALIZACION DE COLUMNAS

In [217]:
def normalizar_columnas(df):
    df.columns = df.columns.str.lower().str.strip()
    return df

In [218]:
ventas_online = normalizar_columnas(ventas_online)
stock = normalizar_columnas(stock)
cotizacion = normalizar_columnas(cotizacion)
cliente = normalizar_columnas(cliente)
ventas_tienda = normalizar_columnas(ventas_tienda)


#VENTAS

##UNIFICAMOS ESTRUCTURA DE VENTAS

####PASO 1 : VERIFICAMOS COLUMNAS EN VENTAS

In [219]:
print(ventas_online.columns)

Index(['venta_id', 'fecha', 'cliente_id', 'producto', 'importe'], dtype='object')


In [220]:
print(ventas_tienda.columns)

Index(['venta_id', 'fecha_venta', 'cliente_id', 'producto_nombre',
       'monto_total'],
      dtype='object')


###PASO 2 : RENOMBRAMOS COLUMNAS PARA UNIFICAR RESULTADOS

In [221]:
ventas_tienda.rename(columns= {
                     'fecha_venta' : 'fecha',
                     'producto_nombre': 'producto',
                     'monto_total': 'importe'}, inplace=True)

### PASO 3 : CREAMOS LA COLUMAS PARA TIPO DE VENTA

In [222]:
ventas_online['tipo_venta'] = 'online'
ventas_tienda['tipo_venta'] = 'tienda'

### PASO 4 : UNIMOS VENTAS



In [223]:
ventas = pd.concat([ventas_tienda, ventas_online], ignore_index=True)
ventas ['etl_warning'] = ''

In [224]:
ventas.sample(5)

,venta_id,fecha,cliente_id,producto,importe,tipo_venta,etl_warning
13,1004,2025-13-01,4.0,Monitor,-300.0,online,
5,2006,2025-01-08,1.0,Mouse,35.0,tienda,
15,1006,2025-01-10,999.0,Mouse,30.0,online,
12,1003,2025/01/07,NaN,Teclado,45.0,online,
8,2009,2025-01-11,5.0,Tablet,410.0,tienda,


## FECHA

### PASO 1 : NULOS Y LOGGIN ERROR

In [225]:
ventas['fecha'] = pd.to_datetime(ventas['fecha'], errors='coerce')
fecha_invalida= ventas['fecha'].isna()
ventas.loc[fecha_invalida, 'etl_warning'] = '|fecha invalida|'

## CLIENTE ID

### PASO 1 : FALTANTES y LOGGIN DE ERROR

creamos faltante para asignarlo como mascara booleana para usarlo como argumento en ventas.loc para el loggin de errores

In [226]:
faltante = ventas['cliente_id'].isna()
ventas.loc[faltante, 'etl_warning'] += '|cliente id faltante|'
ventas.loc[faltante,'cliente_id'] = -1

##PRODUCTO

### PASO 1 : NULO Y LOGGIN ERROR

In [227]:
producto_nulo = ventas['producto'].isna()
ventas.loc[producto_nulo, 'etl_warning'] += '|producto nulo|'


###PASO 2 : VACIO Y LOGGIN ERROR

In [228]:
producto_vacio = ventas['producto'] == ''
ventas.loc[producto_vacio, 'etl_warning'] += '|producto vacio|'

###PASO 3 : NORMALIZACION

In [229]:
ventas['producto'] = ventas['producto'].str.lower().str.strip()

DF

#INTEGRACION DE VENTAS Y CLIENTES

###PASO 1 : UNIMOS POR COLUMNAS CLIENTE_ID

In [230]:
ventas= ventas.merge(cliente, on='cliente_id', how='left')

### PASO 2 : LOGGIN ERROR POR NOMBRE

In [231]:
mask_nombre = ventas['nombre'].isna()
ventas.loc[mask_nombre, 'etl_warning'] += '|nombre faltante|'

###PASO 3 : LOGGIN ERROR POR MAIL

In [232]:
mask_mail = ventas['email'].str.contains('@', na=False)
ventas.loc[~mask_mail, 'etl_warning'] += '|mail invalido|'

# INTEGRAR CON DOLAR

###PASO 1 : NOMALIZAR DOLAR

In [233]:
cotizacion['fecha'] = pd.to_datetime(cotizacion['fecha'], errors='coerce')

###PASO 2 : UNIMOS POR COLUMNA FECHA

In [234]:
ventas = ventas.merge(cotizacion, on='fecha', how='left')

###PASO 3 : LOGGIN ERROR NULOS

In [235]:
mask_nulo = ventas['cotizacion'].isna()
ventas.loc[mask_nulo, 'etl_warning'] += '|cotizacion faltante|'

###PASO 4 : CREAMOS COLUMNA IMPORTE

IMPORTE EN PESOS = IMPORTE * COTIZACION

PARA VALIDACION VERIFICAMOS SI IMPORTE O COTIZACION ES NULO

In [236]:
mask_importe_nulo = ventas['importe'].isna()
mask_cotizacion_nulo = ventas['cotizacion'].isna()
ventas.loc[mask_importe_nulo, 'etl_warning'] += '|importe nulo|'
ventas.loc[mask_cotizacion_nulo, 'etl_warning'] += '|cotizacion nulo|'

In [237]:
ventas['importe_pesos'] = ventas['importe'] * ventas['cotizacion']

In [238]:
ventas["importe_pesos"] = (
    ventas["importe_pesos"]
    .astype(str)
    .str.replace(".", "", regex=False)   # quita separadores de miles
    .str.replace(",", ".", regex=False)  # por si hay coma decimal
)

ventas["importe_pesos"] = pd.to_numeric(ventas["importe_pesos"], errors="coerce")


VALIDAMOS NULIDAD EN IMPRTE PESOS

In [239]:
mask_importe_pesos = ventas['importe_pesos'].isna()
ventas.loc[mask_importe_pesos, 'etl_warning'] += '|importe pesos nulo|'

In [240]:
mask_importe_fuera_rango = ventas["importe_pesos"] > 1e9  # umbral lógico

ventas.loc[mask_importe_fuera_rango, "etl_warning"] += "|importe fuera de rango|"

#VALIDACION DE CALIDAD

In [243]:
print('REGISTROS TOTALES : ', len(ventas))
print('ERRORES DETECTADOS : ', ventas['etl_warning'].notna().sum())
print("\nDETALLES DE ERRORES:")
print(ventas["etl_warning"].value_counts(dropna=False))

REGISTROS TOTALES :  20
ERRORES DETECTADOS :  20

DETALLES DE ERRORES:
etl_warning
                                                                                                                                   7
|fecha invalida||mail invalido||cotizacion faltante||cotizacion nulo||importe pesos nulo|                                          3
|mail invalido|                                                                                                                    2
|nombre faltante||mail invalido|                                                                                                   2
|importe fuera de rango|                                                                                                           1
|cliente id faltante||nombre faltante||mail invalido||cotizacion faltante||cotizacion nulo||importe pesos nulo|                    1
|mail invalido||importe nulo||importe pesos nulo|                                                                      

#GUARDADO DE CAPAS

##DATOS PROCESADOS

REORDENAMOS COLUMNAS PARA QUE APAREZCA AL ULTIMO ETL_WARNING

In [244]:
cols = [c for c in ventas.columns if c != "etl_warning"] + ["etl_warning"]
ventas = ventas[cols]

In [245]:
ventas.to_csv('/content/drive/MyDrive/Varias fuentes/ventas_final.csv', index=False)

##DATASET ANALITICO

In [246]:
ventas.columns

Index(['venta_id', 'fecha', 'cliente_id', 'producto', 'importe', 'tipo_venta',
       'nombre', 'email', 'cotizacion', 'importe_pesos', 'etl_warning'],
      dtype='object')

In [247]:
dataset_final = ventas.groupby(["fecha", "tipo_venta"]).agg({
    "importe_pesos": "sum",
    "venta_id": "count"
}).reset_index()

dataset_final.rename(columns={"venta_id": "cantidad_ventas"}, inplace=True)

dataset_final.to_csv("/content/drive/MyDrive/Varias fuentes/dataset_analitico.csv", index=False)
